We implement the speeding up at no job left behind in the completely online version of the policy, and see if it matches our heuristic.

In [1]:
%matplotlib inline
from pylab import *
import sys
from collections import namedtuple
import random
import time
import numpy as np

In [2]:
def log_normal(mu=0, sigma=1):
    return e**(mu+sigma*np.random.normal())

def exp_samples(lambd=1,n=1):
    """
    generates a numpy array of n samples distributed according to
    an exponential distribution with parameter λ
    """
    random.seed(time.time())
    return (-1/lambd)*log(rand(n))

def exp(lambd=1):
    """
    generates a sample distributed according to
    an exponential distribution with parameter λ
    """
    if lambd < 0:
        return inf()
    random.seed(time.time())
    return (-1/lambd)*log(rand())

def inf():
    return 999999


def printarray(a): 
    print(np.around(a, 3))
    

In [3]:
def gen_interarrivals_and_services(lambd, mu1, n):
    return exp_samples(lambd, n), exp_samples(mu1, n)

In [4]:
def lambd_range(k, g):
    print("Min range %f" % (1-k))
    print("Max range %f" % (1-k+g*k))

def min_threshold(lambd, mu1, mu2, g):
    rho1 = lambd / mu1
    rho2 = lambd / mu2
    return math.ceil(math.log((g*(1-rho2)*rho1) / ((1-rho1)*rho2 - g*(rho2 - rho1)), rho1))

def min_g(lambd, mu1, mu2):
    rho1 = lambd / mu1
    rho2 = lambd / mu2
    return (rho1*rho2 - rho2) / (rho1 - rho2)

In [16]:
k = 0.1
g = .21
mu1 = 1 - k
mu2 = 1
n = 100000


    
# Indicies for counting surge
start = int(0.1 * n)
end = int(0.9 * n)

In [17]:
print(lambd_range(k, g))
lambd = 0.92

Min range 0.900000
Max range 0.921000
None


In [18]:
threshold = min_threshold(lambd, mu1, 1, g)

In [19]:
interarrivals, services = gen_interarrivals_and_services(lambd, mu1, n)

arrival_times = []
for i in range(len(interarrivals)):
    arrival_times.append(sum(interarrivals[0:i + 1]))
#print(arrival_times)

service_times = [arrival_times[0] + services[0]]
for i in range(1, len(services)):
    service_times.append(max(arrival_times[i], service_times[i - 1]) + services[i])
#print(service_times)

In [20]:
def generate_T_N(interarrivals, services):
    T = [0]
    N = [0]

    arrival_index = 0
    service_index = 0
  
    arrival_times = []
    for i in range(len(interarrivals)):
        arrival_times.append(sum(interarrivals[0:i + 1]))
    #print(arrival_times)

    service_times = [arrival_times[0] + services[0]]
    for i in range(1, len(services)):
        service_times.append(max(arrival_times[i], service_times[i - 1]) + services[i])
    #print(service_times)

    next_arrival = arrival_times[arrival_index]
    next_service = service_times[service_index]

    while min(next_arrival, next_service) != inf():
        if next_arrival < next_service:
            N.append(N[-1] + 1)
            T.append(next_arrival)
            arrival_index += 1
            if arrival_index == len(arrival_times):
                next_arrival = inf()
            else:
                next_arrival = arrival_times[arrival_index]
        else:
            N.append(N[-1] - 1)
            T.append(next_service)
            service_index += 1
            if service_index == len(service_times):
                next_service = inf()
            else:
                next_service = service_times[service_index]

    return T, N

In [21]:
# WRONG DO NOT USE

def implement_threshold(interarrvials, services, threshold):
    T = [0]
    N = [0]
    in_surge_num = 0
    in_surge_denom = 0
    S = 0
    start_measuring_surge = 0

    arrival_index = 0
    service_index = 0
  
    arrival_times = []
    for i in range(len(interarrivals)):
        arrival_times.append(sum(interarrivals[0:i + 1]))
    #print(arrival_times)

    service_times = [arrival_times[0] + services[0]]
    for i in range(1, len(services)):
        service_times.append(max(arrival_times[i], service_times[i - 1]) + services[i])
    #print(service_times)

    next_arrival = arrival_times[arrival_index]
    next_service = service_times[service_index]

    while min(next_arrival, next_service) != inf():
        if next_arrival < next_service:
            N.append(N[-1] + 1)
            T.append(next_arrival)
            arrival_index += 1
            if arrival_index == len(arrival_times):
                next_arrival = inf()
            else:
                next_arrival = arrival_times[arrival_index]
        else:
            N.append(N[-1] - 1)
            T.append(next_service)
            service_index += 1
            if T[-1] >= start and T[-1] <= end:
                in_surge_denom += 1
            if service_index == len(service_times):
                next_service = inf()
            else:
                next_service = service_times[service_index]
        # happens once per service index max
        if N[-1] >= threshold:
            services[service_index] = (mu1) * services[service_index]
            if T[-1] >= start and T[-1] <= end:
                in_surge_num += 1
            for i in range(service_index, len(services)):
                service_times[i] = max(arrival_times[i], service_times[i - 1]) + services[i]
            next_service = service_times[service_index]

        ###### SURGE count and proportion #####
        #if T[-1] > 200:
        #    if in_surge == 0 and S == 0: # first time, mark t_temp
        #        start_measuring_surge = T[-2]
        #    t_delta = T[-1] - T[-2]
        #    S = S + (t_delta * N[-2])
        #    if N[-2] >= threshold:
        #        in_surge = in_surge + t_delta

    #S = S / (T[-1] - start_measuring_surge)
    #in_surge = in_surge / (T[-1] - start_measuring_surge)
    return T, N, S, in_surge_num / in_surge_denom

In [25]:
def implement_speedup(interarrivals, services, window):
    w = np.zeros(n) # waiting time array
    index = 0
    
    # Counting the proportion of jobs we speed up, can be no more than g
    speed_up_num = 0
    speed_up_denom = 0
    start = time.time()

    for index in range(n - 1):
        #lookahead_range = 0
        for i in range(index, min(index + window, n - 1)):
            #lookahead_range += min(services[i] - interarrivals[i], services[i])
            w[i + 1] = max(w[i] + services[i] - interarrivals[i], 0)
            if w[i + 1] <= 0:
                #print("Not speeding up job at T=%.2f" % (arrival_times[index]))
                break
            # if we reach the end or it comes back down within a certain window
            if i == min(index + window - 1, n - 2):
                services[index] = (mu1) * services[index]
                if index >= start and index <= end:
                    speed_up_num += 1
                break
                    #print("Speeding up job at T=%.2f" % (arrival_times[index]))
        if index >= start and index <= end:
            speed_up_denom += 1

        w[index + 1] = max(w[index] + services[index] - interarrivals[index], 0)
        
        if index % 10000 == 0:
            now = time.time()
            print("Done with %d trials in %.3f s" % (index, now - start))
    
    T2, N2 = generate_T_N(interarrivals, services)
    return T2, N2, w, speed_up_num / speed_up_denom

In [28]:
# implement speedup policy, except speed up the last 10% of patients because
# we are approaching the end of the simulation horizon
def implement_speedup_2(interarrivals, services, window):
    w = np.zeros(n) # waiting time array
    index = 0
    
    # Counting the proportion of jobs we speed up, can be no more than g
    speed_up_num = 0
    speed_up_denom = 0

    for index in range(n - 1):
        #lookahead_range = 0
        if index == end:
            print(speed_up_num / speed_up_denom)
        if index >= end:
            services[index] = (mu1) * services[index]
            if index >= start and index <= n:
                speed_up_num += 1
        else: 
            for i in range(index, min(index + window, n - 1)):

                #lookahead_range += min(services[i] - interarrivals[i], services[i])
                w[i + 1] = max(w[i] + services[i] - interarrivals[i], 0)
                if w[i + 1] <= 0:
                    #print("Not speeding up job at T=%.2f" % (arrival_times[index]))
                    break
                # if we reach the end or it comes back down within a certain window
                if i == min(index + window - 1, n - 2):
                    services[index] = (mu1) * services[index]
                    if index >= start and index <= n:
                        speed_up_num += 1
                        #print("Speeding up job at T=%.2f" % (arrival_times[index]))
        if index >= start and index <= n:
            speed_up_denom += 1

        w[index + 1] = max(w[index] + services[index] - interarrivals[index], 0)
    
    T2, N2 = generate_T_N(interarrivals, services)
    return T2, N2, w, speed_up_num / speed_up_denom

In [30]:
print("k = %.2f" % (k))
print("g = %.2f" % (g))
print("\n")
T, N = generate_T_N(interarrivals.copy(), services.copy())
'''T_threshold, N_threshold, S, in_surge = implement_threshold(interarrivals.copy(), services.copy(), threshold)
print("Threshold policy with threshold %d" % (threshold))
print("Threshold speed up for %.2f proportion of time" % (in_surge))
print("Threshold avg queue length %.2f" % (np.mean(N_threshold[start:end])))
print("Threshold average Q %.2f" % (S))'''
'''print("\n")
T_njlb, N_njlb, w, in_surge_2 = implement_speedup(interarrivals.copy(), services.copy(), 10000, .4)
print("NJLB policy with infinite lookahead, surge 0.4 of time when lookahead conditions are not met")
print("NJLB speed up for %.2f proportion of time" % (in_surge_2))
print("NJLB avg queue length %.2f" % (np.mean(N_njlb[start:end])))
print("NJLB Expected waiting time %.2f" % (np.mean(w)))'''

print("\n")
T_njlb3, N_njlb3, w3, in_surge_23 = implement_speedup(interarrivals.copy(), services.copy(), 100000)
print("NJLB policy with infinite lookahead, surge all of time when lookahead conditions are not met")
print("NJLB speed up for %.2f proportion of time" % (in_surge_23))
print("NJLB Expected waiting time %.2f" % (np.mean(w3[start:end])))

'''print("\n")

T_njlb2, N_njlb2, w2, in_surge_22 = implement_speedup_2(interarrivals.copy(), services.copy(), 10000)
print("NJLB policy with infinite lookahead, surge last 10% of jobs")
print("NJLB speed up for %.2f proportion of time" % (in_surge_22))
print("NJLB Expected waiting time %.2f" % (np.mean(w2)))
'''

k = 0.10
g = 0.21




Done with 0 trials in 0.000 s
Done with 10000 trials in 423.315 s
Done with 20000 trials in 716.752 s
Done with 30000 trials in 897.475 s
Done with 40000 trials in 1209.891 s
Done with 50000 trials in 1459.949 s
Done with 60000 trials in 1625.057 s
Done with 70000 trials in 1764.764 s
Done with 80000 trials in 1911.254 s
Done with 90000 trials in 1993.959 s


TypeError: slice indices must be integers or None or have an __index__ method

In [ ]:

figure(figsize=(15,6))
plot(T[:180000], N[:180000], label='Sample Path')
#plot(T_threshold[:180000], N_threshold[:18000], label='Sample Path with threshold %d' % (threshold))
#plot(T_njlb[:18000], N_njlb[:18000], label='Sample Path with no job left behind, surge 40%')
plot(T_njlb3[:180000], N_njlb3[:180000], label='Sample Path with no job left behind surge normally')
#plot(T_njlb2[:18000], N_njlb2[:18000], label='Sample Path with no job left behind surge normally and last 10%')
xlabel('Time')
ylabel('Number in Queue')
title("Speed Up Policies")
legend()

In [53]:

T_njlb2, N_njlb2, w2, in_surge_22 = implement_speedup(interarrivals.copy(), services.copy(), 10, 1)
print("NJLB speed up for %.2f proportion of time" % (in_surge_22))
print("NJLB Expected waiting time %.2f" % (np.mean(w2)))

NJLB speed up for 0.77 proportion of time
NJLB Expected waiting time 10.28


In [54]:

T_njlb2, N_njlb2, w2, in_surge_22 = implement_speedup(interarrivals.copy(), services.copy(), 100, 1)
print("NJLB speed up for %.2f proportion of time" % (in_surge_22))
print("NJLB Expected waiting time %.2f" % (np.mean(w2)))

NJLB speed up for 0.49 proportion of time
NJLB Expected waiting time 11.60


In [ ]:

T_njlb2, N_njlb2, w2, in_surge_22 = implement_speedup(interarrivals.copy(), services.copy(), 10000, 1)
print("NJLB speed up for %.2f proportion of time" % (in_surge_22))
print("NJLB Expected waiting time %.2f" % (np.mean(w2)))

In [ ]:

T_njlb, N_njlb, w, in_surge_2 = implement_speedup_2(interarrivals.copy(), services.copy(), 10000, 1)
print("NJLB speed up for %.2f proportion of time" % (in_surge_2))
print("NJLB Expected waiting time %.2f" % (np.mean(w)))